# Series 04. Part I - Convolutional Neural Network

Welcome to the series. This is the first part of tutorial series #04. The overview of tutorial series #04 is as follows:

- **Part I  :** Convolutional Neural Network (this notebook)
- **Part II :** Hands-on with Kannada MNIST dataset using Pytorch [[notebook](#04.b.Kaggle_Kannada_Pytorch)]
- **Part III:** Pytorch model training and deployment in AWS Sagemaker [[notebook](#04.b.Kaggle_Kannada_Pytorch)]

## Table of Content
1. [__CONVOLUTIONAL NEURAL NETWORKS__](#100)
    1. [__Convolution Layer__](#110)
        1. [Steps of Convolution Operation](#111)
    2. [__Padding and Stride__](#120)
        1. [Padding](#121)
        1. [Stride](#122)
    3. [__Pooling Layer__](#130)
        1. [Types of Pooling](#131)
        1. [Why we use pooling layer](#132)
    4. [__Batch Normalization Layer__](#140)
        1. [What is batch norm](#141)
        1. [Batch norm algorithm](#142)
    5. [__Flattening Layer__](#150)
    6. [__Summary__](#160)
    
2. [__REFERENCES__](#200)

## __CONVOLUTIONAL NEURAL NETWORKS__ <a id="200"></a>

Convolutional neural network (CNN) is a type of neural network architecture specially made to deal with visual data like images and videos. In this article we will discuss the architecture of CNN and implement it on [Kannada MNIST]() dataset in the [second part](04.b.Kaggle_Kannada_Pytorch.ipynb) of this tutorial. The main benefit of using a CNN over simple ANN on visual data is that CNN’s are constrained to deal with image data exclusively. Two main features of CNNs are

- Weight sharing
- Feature extractors

![](https://i.imgur.com/RYMoJpL.png)

As we described above, a simple ConvNet is a sequence of layers, and every layer of a ConvNet transforms one volume of activations to another through a differentiable function. We use three main types of layers to build ConvNet architectures: Convolutional Layer, Pooling Layer, and Fully-Connected Layer (exactly as seen in regular Neural Networks). We will stack these layers to form a full ConvNet architecture.

Several new layers are introduced in CNNs to extract the useful features from our image or reducing the size of image without using the original representation.


### 1. Convolution Layer
Convolutional layer apply convolution operation on the input layer, passing the results to next layer. A convolution operation is basically computing a dot product between their weights and a small region they are connected(currently overlapping) to in the input volume. This will change the dimensions depending on the filter size used and number of filters used.
![](images/convlayer.png)


You can think of left one as the “input” signal (or image), and the other (called the kernel) as a “filter” on the input image, producing an output image (so convolution takes two matrix as input and produces a third as output). 

#### **Steps of convolution operation** <a id="111"></a> 
1. Take the element wise production of the upper left corner sub-matrix and filter
2. Then sum all row and column to obtain a single value
3. Result is the upper corner cell's value of your new output matrix. The first convolution in above example yield output $3$.
4. After that, slide the filter over do the same thing.

![](images/convolution_2d.gif)

### 2. Stride and Padding  <a id="112"></a>

In general, assuming the input shape is  $n_h×n_w$  and the convolution kernel window shape is  $k_h×k_w$, then the output shape will be

$$(n_h−k_h+1)×(n_w−k_w+1).$$
 
Therefore, the output shape of the convolutional layer is determined by the shape of the input and the shape of the convolution kernel window.

In several cases we might want to incorporate particular techniques—padding and strides, regarding the size of the output:

- In general, since kernels generally have width and height greater than  1 , that means that after applying many successive convolutions, we will wind up with an output that is much smaller than our input. If we start with a  240×240  pixel image,  10  layers of  5×5  convolutions reduce the image to  200×200  pixels, slicing off  30%  of the image and with it obliterating any interesting information on the boundaries of the original image. Padding handles this issue.
- In some cases, we want to reduce the resolution drastically if say we find our
- original input resolution to be unwieldy. Strides can help in these instances.

#### 2.1 Padding

Now you might notice that convolutional operation leave a gap in the border of the output result. This means that the boundary of the image input won't be traversed by the convolution kernel. The problem here is that as our convolutional network get deeper and deeper, the input to the next layer get shrinked and we lose a lot of information on the way.


Padding can help solve the issue of shrinking input. Padding is used to preserve the boundary information , since without padding they are only traversed once.

<img src="https://i.imgur.com/MWorvXh.png" width=800>

In general, if we add a total of $p_h$  rows of padding (roughly half on top and half on bottom) and a total of $p_w$  columns of padding (roughly half on the left and half on the right), the output shape will be

$$(n_h−k_h+p_h+1)×(n_w−k_w+p_w+1). $$

This means that the height and width of the output will increase by  $p_h$  and  $p_w$  respectively.

In many cases, we will want to set  $p_h=k_h−1$  and  $p_w=k_w−1$  to give the input and output the same height and width. This is what we call "same padding"

This will make it easier to predict the output shape of each layer when constructing the network. Assuming that  kh  is odd here, we will pad  ph/2  rows on both sides of the height. If  kh  is even, one possibility is to pad  ⌈ph/2⌉  rows on the top of the input and  ⌊ph/2⌋  rows on the bottom. We will pad both sides of the width in the same way.

#### 2.2 Stride
When performing convolution,  we start with the convolution window at the top-left corner of the input array, and then slide it over all locations both down and to the right. In previous examples, we default to sliding one pixel at a time. However, sometimes, either for computational efficiency or because we wish to downsample, we move our window more than one pixel at a time, skipping the intermediate locations.

We refer to the number of rows and columns traversed per slide as the *stride*. So far, we have used strides of  1 , both for height and width. Sometimes, we may want to use a larger stride. 

![](images/stride.png)

The figure above shows a two-dimensional cross-correlation operation with a stride of  3  vertically and  2  horizontally. We can see that when the second element of the first column is output, the convolution window slides down three rows. The convolution window slides two columns to the right when the second element of the first row is output.

In general, when the stride for the height is  sh  and the stride for the width is  sw , the output shape is

$$
⌊(n_h−k_h+p_h+s_h)/s_h⌋×⌊(n_w−k_w+p_w+s_w)/s_w⌋.
$$
 
If we set  $p_h=k_h−1$  and  $p_w=k_w−1$ , then the output shape will be simplified to  $⌊(n_h+s_h−1)/s_h⌋×⌊(n_w+s_w−1)/s_w⌋$ . Going a step further, if the input height and width are divisible by the strides on the height and width, then the output shape will be  $(n_h/s_h)×(n_w/s_w)$ .

### 3. Pooling Layer
Pooling layer will perform a down-sampling operation along the width and resulting in the reduction of the dimensions. The sole purpose of pooling is to reduce spatial dimensions. There are various types of pooling in which the most common is Max Pooling, i.e taking the maximum element from the window.
![](https://i.imgur.com/5Q54piT.png)

#### Why do we use Pooling? <a id="152"></a>

1. To gain robustness on feature extraction;
    * Pooling prevents the model from over-training by discarding the unnecessary data relative to the selected value.
2. To speed up the computation;
    * Reducing the size of the representation for increasing the speed of the computation.

#### Types of Pooling <a id="153"></a>
    
* **Max Pooling**: The operation of selecting the maximum value from each mini-group (pool) as the picture above
* **Average Pooling**: The operation of selecting the average value from each mini-group.

### 4. Batch Normalization <a id="160"></a>

#### **What is Batch Norm?** <a id="161"></a>

* Batch Normalization is a method used to normalize the input of a hidden layer by adjusting and scaling the activations.
* The method consists of performing normalization on each neuron of a specific layer on each training mini-batch.
* Normalized neurons of $l$'th layer will have a specific mean and variance for the problem according to trained $\gamma$ and $\beta$ parameters.


The algorithm of the batch norm is shown below;

![batch-norm.png](images/batch-norm.png)

### Flattening Layer
This layer will convert the 3-dimensions (height,width,depth) into a single long vector to feed it to the fully connected layer or Dense layer. It connects every neuron in one layer to every neuron in another layer.

Fully Connected Layer and Output Layer
Fully connected layers or dense layers are the same hidden layers consisting of defined number of neurons connected with elements of another layer that we discussed in simple ANN. However the output layer is also the same but the number of neurons depend on our task. For instance in CIFAR-10 dataset we have 10 classes hence we will have 10 neurons in the outer layer.

### Summary
![](https://i.imgur.com/x0DIK9L.jpg)

In summary, the architecture of CNN , we can simply understand that it consist of an input layer followed by a Conv layer. The dimensions of conv layer depends on the data and problem, hence changing the dimensions accordingly. After the Conv Layer there is a activation layer , usually ReLU since it gives better results. After some conv and relu combination , pooling layer is used to reduce the size. Then after some combination of previously defined architecture , flattening layer is used to flatten the input for fully connected layer. Next to these layer, the last layer is the output layer.

## References 

[1] Donahue, J, et al. "Long-term recurrent convolutional networks for visual recognition and description."     Proceedings of the IEEE conference on computer vision and pattern recognition. 2015.

[2]Vinyals, Oriol, et al. "Show and tell: Lessons learned from the 2015 mscoco image captioning challenge." IEEE transactions on pattern analysis and machine intelligence 39.4 (2017): 652-663.

[3] TensorFlow Show and Tell:A Neural Image Caption Generator [example] (https://github.com/tensorflow/models/tree/master/im2txt)

[4] Karapthy, A. [NeuralTalk2](https://github.com/karpathy/neuraltalk2)

[5]Lin, Tsung-Yi, et al. "Microsoft coco: Common objects in context." European Conference on Computer Vision. Springer International Publishing, 2014.